## Step 1: Environment Setup

In [ ]:
# Clone repository
!git clone https://github.com/viraj1011JAIN/tri-objective-robust-xai-medimg.git
%cd tri-objective-robust-xai-medimg

In [ ]:
# Check GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Training will be slow.")

In [ ]:
# Install dependencies (this may take 2-3 minutes)
!pip install -r requirements.txt --quiet
!pip install pytest pytest-cov --quiet

print("✓ Dependencies installed")

In [ ]:
# Fix Pillow compatibility issue (scikit-image requires >=10.1)
!pip uninstall -y pillow 2>&1 | grep -E "(Successfully|Found)" || true
!pip install pillow==10.1.0 --no-warn-conflicts

print("✓ Pillow compatibility fixed")

In [ ]:
# Fix pyarrow compatibility issue (required for mlflow and other packages)
!pip uninstall -y pyarrow 2>&1 | grep -E "(Successfully|Found)" || true
!pip install pyarrow==15.0.2 --quiet

print("✓ PyArrow fixed")
print("🔄 Restarting runtime to load new pyarrow...")

# Auto-restart runtime
import os
os.kill(os.getpid(), 9)

**⚠️ IMPORTANT:** If you get a pyarrow error in the next cell, **restart the runtime** (`Runtime → Restart runtime`) and then **continue from cell 7 (skip cells 1-6)**.

In [ ]:
# Verify core imports
import sys
sys.path.insert(0, '/content/tri-objective-robust-xai-medimg')

try:
    import torch
    import torchvision
    import timm
    import mlflow
    import yaml
    print("✓ All core packages imported successfully")
    print(f"  - torch: {torch.__version__}")
    print(f"  - torchvision: {torchvision.__version__}")
    print(f"  - timm: {timm.__version__}")
except ImportError as e:
    print(f"✗ Import error: {e}")

## Step 2: Verify Project Structure

In [ ]:
# Check critical files exist
from pathlib import Path
import os

# Ensure we're in the correct directory
if not os.path.exists('src'):
    os.chdir('/content/tri-objective-robust-xai-medimg')
    print("📁 Changed to project directory")

def check_file(path, description):
    exists = Path(path).exists()
    symbol = "✓" if exists else "✗"
    print(f"{symbol} {description}: {path}")
    return exists

print("=" * 60)
print("PHASE 3 FILE STRUCTURE CHECK")
print("=" * 60)
print(f"Current directory: {os.getcwd()}")
print()

checks = [
    ("src/models/__init__.py", "Models package"),
    ("src/models/build.py", "Model factory"),
    ("src/models/base_model.py", "Base model class"),
    ("src/models/resnet.py", "ResNet50"),
    ("src/models/efficientnet.py", "EfficientNet"),
    ("src/losses/task_loss.py", "Task losses"),
    ("src/datasets/__init__.py", "Datasets package"),
    ("src/training/baseline_trainer.py", "Baseline trainer"),
    ("tests/unit/test_models.py", "Model tests"),
]

all_exist = all(check_file(path, desc) for path, desc in checks)

print("=" * 60)
if all_exist:
    print("✓ All critical files present")
else:
    print("✗ Some files missing - check repository")

## Step 3: Test Model Registry & Build System

In [ ]:
# Import model building functions
from src.models import build_model
from src.models.build import MODEL_REGISTRY
import torch

print("=" * 60)
print("MODEL REGISTRY CHECK")
print("=" * 60)

# List available models
print("\nAvailable models in registry:")
for model_name in MODEL_REGISTRY.keys():
    print(f"  - {model_name}")

# Test building each model
print("\nTesting model instantiation:")
test_models = ['resnet50', 'efficientnet_b0']
results = {}

for model_name in test_models:
    try:
        # Build model
        model = build_model(model_name, num_classes=7)
        print(f"\n✓ {model_name}: Successfully built")
        
        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        
        # Test forward pass
        model.eval()
        with torch.no_grad():
            dummy_input = torch.randn(2, 3, 224, 224)
            output = model(dummy_input)
            print(f"  Input shape: {tuple(dummy_input.shape)}")
            print(f"  Output shape: {tuple(output.shape)}")
            
            # Verify output shape
            assert output.shape == (2, 7), f"Expected (2, 7), got {output.shape}"
            assert torch.isfinite(output).all(), "Output contains NaN or Inf"
            print(f"  ✓ Forward pass successful")
        
        results[model_name] = "PASS"
        
    except Exception as e:
        print(f"\n✗ {model_name}: FAILED")
        print(f"  Error: {str(e)}")
        results[model_name] = "FAIL"

# Summary
print("\n" + "=" * 60)
print("MODEL BUILD SUMMARY")
print("=" * 60)
for model_name, status in results.items():
    symbol = "✓" if status == "PASS" else "✗"
    print(f"{symbol} {model_name}: {status}")

all_passed = all(status == "PASS" for status in results.values())
if all_passed:
    print("\n✓✓✓ All models working correctly ✓✓✓")
else:
    print("\n✗✗✗ Some models failed ✗✗✗")

## Step 4: Test Loss Functions

In [ ]:
# Test loss function implementations
from src.losses.task_loss import FocalLoss
import torch
import torch.nn as nn

print("=" * 60)
print("LOSS FUNCTION TESTS")
print("=" * 60)

# Test data (predictions need requires_grad=True)
batch_size = 8
num_classes = 7
predictions = torch.randn(batch_size, num_classes, requires_grad=True)
targets = torch.randint(0, num_classes, (batch_size,))

losses_to_test = [
    ("CrossEntropyLoss", nn.CrossEntropyLoss()),
    ("FocalLoss", FocalLoss(num_classes=num_classes, alpha=1.0, gamma=2.0)),
]

results = {}

for loss_name, loss_fn in losses_to_test:
    try:
        # Compute loss
        loss = loss_fn(predictions, targets)
        
        print(f"\n✓ {loss_name}")
        print(f"  Loss value: {loss.item():.4f}")
        
        # Verify properties
        assert loss.item() > 0, "Loss should be positive"
        assert torch.isfinite(loss), "Loss should be finite"
        assert loss.requires_grad, "Loss should require gradients"
        
        # Test backward pass
        loss.backward()
        print(f"  ✓ Backward pass successful")
        
        results[loss_name] = "PASS"
        
        # Reset gradients for next test
        predictions.grad = None
        
    except Exception as e:
        print(f"\n✗ {loss_name}: FAILED")
        print(f"  Error: {str(e)}")
        results[loss_name] = "FAIL"

# Summary
print("\n" + "=" * 60)
print("LOSS FUNCTION SUMMARY")
print("=" * 60)
for loss_name, status in results.items():
    symbol = "✓" if status == "PASS" else "✗"
    print(f"{symbol} {loss_name}: {status}")

if all(status == "PASS" for status in results.values()):
    print("\n✓✓✓ All loss functions working ✓✓✓")
else:
    print("\n✗✗✗ Some loss functions failed ✗✗✗")

## Step 5: Test Dataset Infrastructure

In [ ]:
# Test dataset loading infrastructure
from src.datasets.transforms import get_train_transforms, get_test_transforms
import os

print("=" * 60)
print("DATASET INFRASTRUCTURE TEST")
print("=" * 60)

try:
    # Test transform functions
    print("\nTesting transform functions...")
    train_transforms = get_train_transforms(image_size=224)
    test_transforms = get_test_transforms(image_size=224)
    
    print("✓ Train transforms created")
    print(f"  Type: {type(train_transforms)}")
    print("✓ Test transforms created")
    print(f"  Type: {type(test_transforms)}")
    
    # Test transform on dummy image
    from PIL import Image
    import numpy as np
    
    # Create dummy image as numpy array (required for Albumentations)
    dummy_image = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)
    
    # Apply transform using Albumentations syntax (named argument)
    transformed = train_transforms(image=dummy_image)
    transformed_tensor = transformed['image']
    
    print(f"\n✓ Transform applied to dummy image")
    print(f"  Output shape: {tuple(transformed_tensor.shape)}")
    print(f"  Output dtype: {transformed_tensor.dtype}")
    print(f"  Value range: [{transformed_tensor.min():.3f}, {transformed_tensor.max():.3f}]")
    
    assert transformed_tensor.shape == (3, 224, 224), f"Expected (3, 224, 224), got {transformed_tensor.shape}"
    
    print("\n✓✓✓ Dataset infrastructure working ✓✓✓")
    
except Exception as e:
    print(f"\n✗ Dataset infrastructure test FAILED")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

## Step 6: Run Unit Tests

In [ ]:
# Run pytest suite for Phase 3 components
!pytest tests/unit/test_models.py -v --tb=short --color=yes

## Step 7: Training Infrastructure Check

In [ ]:
# Test training infrastructure imports
print("=" * 60)
print("TRAINING INFRASTRUCTURE CHECK")
print("=" * 60)

components = []

try:
    from src.training.baseline_trainer import BaselineTrainer
    print("✓ BaselineTrainer imported")
    components.append(("BaselineTrainer", "PASS"))
except Exception as e:
    print(f"✗ BaselineTrainer import failed: {e}")
    components.append(("BaselineTrainer", "FAIL"))

try:
    from src.training.base_trainer import BaseTrainer
    print("✓ BaseTrainer imported")
    components.append(("BaseTrainer", "PASS"))
except Exception as e:
    print(f"✗ BaseTrainer import failed: {e}")
    components.append(("BaseTrainer", "FAIL"))

try:
    import torch.optim as optim
    print("✓ Optimizers available")
    components.append(("Optimizers", "PASS"))
except Exception as e:
    print(f"✗ Optimizers import failed: {e}")
    components.append(("Optimizers", "FAIL"))

# Summary
print("\n" + "=" * 60)
print("TRAINING INFRASTRUCTURE SUMMARY")
print("=" * 60)
for component, status in components:
    symbol = "✓" if status == "PASS" else "✗"
    print(f"{symbol} {component}: {status}")

if all(status == "PASS" for _, status in components):
    print("\n✓✓✓ Training infrastructure ready ✓✓✓")
else:
    print("\n✗✗✗ Some components missing ✗✗✗")

## Step 8: Complete Phase 3 Status Report

In [ ]:
# Generate comprehensive Phase 3 status report
from pathlib import Path

def check_component(path, name):
    exists = Path(path).exists()
    return (name, "✓" if exists else "✗", exists)

print("=" * 70)
print("PHASE 3: MODEL ARCHITECTURE & BASELINE - STATUS REPORT")
print("=" * 70)

sections = {
    "3.1 Model Architecture": [
        ("src/models/base_model.py", "Base model abstract class"),
        ("src/models/resnet.py", "ResNet50Classifier"),
        ("src/models/efficientnet.py", "EfficientNetB0Classifier"),
        ("src/models/build.py", "Model factory"),
        ("src/models/model_registry.py", "Model registry"),
    ],
    "3.2 Loss Functions": [
        ("src/losses/task_loss.py", "Task loss implementations"),
        ("src/losses/calibration_loss.py", "Calibration losses"),
    ],
    "3.3 Training Infrastructure": [
        ("src/training/base_trainer.py", "Base trainer"),
        ("src/training/baseline_trainer.py", "Baseline trainer"),
        ("scripts/training/train_baseline.py", "Training script"),
    ],
    "3.4-3.6 Experiment Configs": [
        ("configs/experiments/baseline.yaml", "Baseline config"),
        ("configs/datasets/isic.yaml", "ISIC dataset config"),
    ],
    "3.7 Fairness & Evaluation": [
        ("src/evaluation/fairness.py", "Fairness metrics"),
    ],
    "3.8 Testing": [
        ("tests/unit/test_models.py", "Model unit tests"),
        ("tests/unit/test_losses.py", "Loss function tests"),
    ],
}

section_results = {}

for section_name, items in sections.items():
    print(f"\n{section_name}")
    print("-" * 70)
    
    section_passed = 0
    section_total = len(items)
    
    for path, desc in items:
        name, symbol, exists = check_component(path, desc)
        print(f"  {symbol} {desc}")
        if exists:
            section_passed += 1
    
    section_results[section_name] = (section_passed, section_total)

# Overall summary
print("\n" + "=" * 70)
print("OVERALL PHASE 3 SUMMARY")
print("=" * 70)

total_passed = sum(p for p, _ in section_results.values())
total_items = sum(t for _, t in section_results.values())

for section, (passed, total) in section_results.items():
    percentage = (passed / total * 100) if total > 0 else 0
    status = "✓" if passed == total else "⚠" if passed > 0 else "✗"
    print(f"{status} {section}: {passed}/{total} ({percentage:.0f}%)")

overall_percentage = (total_passed / total_items * 100) if total_items > 0 else 0
print("\n" + "=" * 70)
print(f"PHASE 3 COMPLETION: {total_passed}/{total_items} ({overall_percentage:.1f}%)")
print("=" * 70)

if overall_percentage >= 90:
    print("\n✓✓✓ PHASE 3 READY FOR TRAINING ✓✓✓")
elif overall_percentage >= 70:
    print("\n⚠⚠⚠ PHASE 3 MOSTLY COMPLETE - SOME COMPONENTS MISSING ⚠⚠⚠")
else:
    print("\n✗✗✗ PHASE 3 INCOMPLETE - MAJOR COMPONENTS MISSING ✗✗✗")

## Next Steps

If all tests pass:

1. **Mount Google Drive** (for persistent storage):
   ```python
   from google.colab import drive
   drive.mount('/content/drive')
   ```

2. **Download datasets** to Colab or Drive

3. **Run baseline training**:
   ```bash
   !python scripts/training/train_baseline.py --config configs/experiments/baseline.yaml
   ```

4. **Monitor with MLflow** or TensorBoard

If tests fail, check:
- Repository clone was successful
- All dependencies installed correctly
- GPU runtime is enabled
- Python path includes project root